<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=15aeb8c24fc6f167011940eabfa3cc772158e720edf4e17fd139a95444587300
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-06 13:47:08
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.44 C
-------------------
Today PnL: -19.30 K (-0.13%)
Current PnL: -25.13 L (-16.02%)
CY Booked + Current PnL: -10.84 L (-6.91%)
-------------------
Total profit:  2.01 L
Total loss:  -27.13 L
-------------------
Total Booked + Current PnL: 16.14 L (12.43%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.93%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 90.09 L (62.59%)
Deployed:  1.30 C
Current:  1.44 C
CAGR/XIRR %: 5.67%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,0.83,16.49,2.75,19.69,6143.0,31623.0,223365.0,5.14,71.0,X-MC,3.61,78.0,5.15,1.55,24.41,XY25,NTT,AC
51,NESTLEIND,1377.00,0.46,16.43,4.26,21.40,13165.0,43618.0,309044.0,7.98,71.0,X-LC,5.02,12.0,3.31,2.15,23.80,XY25,NTT,FMCG
49,MASFIN,398.61,5.43,3.48,17.95,22.05,18199.0,3405.0,101385.0,-10.57,64.0,H-SC,6.61,164.0,0.19,0.70,47.37,XR,ATH,FINANCE
77,TTKPRESTIG,770.00,-0.90,-19.37,24.13,0.09,19608.0,-19518.0,81259.0,88.69,44.0,M-SC,3.26,253.0,-1.00,0.56,6.76,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-0.00,5.11,13.42,19.23,21225.0,7696.0,158156.0,22.81,83.0,L-LC,5.66,179.0,0.36,1.10,27.48,XY25,ATH,MINING


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.10,9.27,-25.80,88.17,39.61,67053.0,-26450.0,76050.0,-49.88,67.0,H-SC,32.26,135.0,-0.39,0.53,23.10,AR,ATH,MISC
49,MASFIN,398.61,5.43,3.48,17.95,22.05,18199.0,3405.0,101385.0,-10.57,64.0,H-SC,6.61,164.0,0.19,0.70,47.37,XR,ATH,FINANCE
0,5PAISA,593.00,2.77,-26.51,53.63,12.90,86117.0,-57920.0,160576.0,136.78,72.0,H-SC,10.98,161.0,-0.67,1.12,33.52,OX40N,NTT,FINANCE
40,INDUSINDBK,1800.00,2.40,-31.96,95.32,32.89,50070.0,-24678.0,52528.0,-748.59,74.0,L-MC,7.24,259.0,-0.49,0.36,44.66,XR,NTT,BANKS
54,QUESS,424.00,2.31,-26.57,94.53,42.84,45122.0,-17273.0,47733.0,-52.07,56.0,X-SC,29.21,83.0,-0.38,0.33,7.99,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1952.00,-4.68,-3.44,29.77,25.30,57766.0,-6921.0,194042.0,-6.52,38.0,X-LC,8.96,17.0,-0.12,1.35,29.98,XY25,BTT,REFINERIES
83,WHIRLPOOL,2270.00,-3.01,-28.83,155.63,81.92,127143.0,-33102.0,81696.0,-58.28,32.0,M-SC,24.71,236.0,-0.26,0.57,0.00,XR,NTT,DURABLES
74,TITAGARH,1548.00,-2.56,-18.97,79.85,45.74,156698.0,-45937.0,196240.0,7.85,54.0,H-SC,2.97,174.0,-0.29,1.36,27.63,XY24,NTT,ENGINEERING
43,ITC,452.00,-1.92,-15.93,31.78,10.78,64204.0,-38291.0,202027.0,-49.73,13.0,X-LC,4.25,5.0,-0.60,1.40,0.00,X40,NTT,FMCG
29,HAPPSTMNDS,1480.71,-1.74,-43.21,225.50,84.85,164128.0,-55381.0,72784.0,-32.49,35.0,H-SC,14.69,147.0,-0.34,0.51,0.17,AR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.17,-0.79,109.57,107.91,160495.0,-1167.0,146477.0,-19.34,57.0,M-SC,10.22,216.0,-0.01,1.02,9.22,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.17,-0.79,109.57,107.91,160495.0,-1167.0,146477.0,-19.34,57.0,M-SC,10.22,216.0,-0.01,1.02,9.22,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.36,-16.98,20.48,0.02,29679.0,-29640.0,144919.0,120.98,47.0,M-SC,7.20,234.0,-1.00,1.01,26.05,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.90,-19.37,24.13,0.09,19608.0,-19518.0,81259.0,88.69,44.0,M-SC,3.26,253.0,-1.00,0.56,6.76,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.35,-21.21,44.02,13.47,93530.0,-57195.0,212472.0,-67.53,53.0,H-SC,5.38,158.0,-0.61,1.48,8.04,XY24,NTT,PAINTS
66,SIS,528.00,-0.09,-22.37,56.21,21.26,48447.0,-24842.0,86190.0,2035.75,57.0,H-SC,4.28,166.0,-0.51,0.60,16.45,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.21,3.36,16.82,20.74,35320.0,6820.0,209990.0,35.83,73.0,M-MC,6.87,189.0,0.19,1.46,14.33,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.00,5.11,13.42,19.23,21225.0,7696.0,158156.0,22.81,83.0,L-LC,5.66,179.0,0.36,1.10,27.48,XY25,ATH,MINING
84,WIPRO,420.00,0.36,8.38,58.94,72.27,108246.0,14206.0,183654.0,-6.99,55.0,M-LC,5.80,99.0,0.13,1.28,15.08,XR,NTT,IT
49,MASFIN,398.61,5.43,3.48,17.95,22.05,18199.0,3405.0,101385.0,-10.57,64.0,H-SC,6.61,164.0,0.19,0.70,47.37,XR,ATH,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,MASFIN,398.61,5.43,3.48,17.95,22.05,18199.0,3405.0,101385.0,-10.57,64.0,H-SC,6.61,164.0,0.19,0.70,47.37,XR,ATH,FINANCE
84,WIPRO,420.00,0.36,8.38,58.94,72.27,108246.0,14206.0,183654.0,-6.99,55.0,M-LC,5.80,99.0,0.13,1.28,15.08,XR,NTT,IT
25,FINCABLES,1641.55,0.17,-0.79,109.57,107.91,160495.0,-1167.0,146477.0,-19.34,57.0,M-SC,10.22,216.0,-0.01,1.02,9.22,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,0.71,-2.09,38.52,35.63,79042.0,-4376.0,205198.0,-15.85,54.0,H-MC,2.97,119.0,-0.06,1.43,14.24,AR,ATH,PHARMA
38,INDIAMART,4810.62,-1.36,-6.78,121.76,106.72,139991.0,-8363.0,114973.0,-53.87,37.0,H-SC,6.87,139.0,-0.06,0.80,15.01,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-1.92,-15.93,31.78,10.78,64204.0,-38291.0,202027.0,-49.73,13.0,X-LC,4.25,5.0,-0.60,1.40,0.00,X40,NTT,FMCG
22,DIXON,18931.72,-1.55,-17.74,59.57,31.26,113078.0,-40944.0,189824.0,-61.61,29.0,X-MC,18.00,56.0,-0.36,1.32,0.82,X40N,ATH,IT
23,DMART,5391.45,0.64,-11.79,46.93,29.60,103326.0,-29430.0,220170.0,-24.77,31.0,X-LC,9.68,19.0,-0.28,1.53,7.80,X40N,ATH,FMCG
7,AWL,485.00,-0.72,-24.77,108.95,57.19,267298.0,-80787.0,245340.0,-64.40,31.0,X-MC,8.09,58.0,-0.30,1.70,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.08,-0.68,-10.63,46.15,30.62,97773.0,-25194.0,211860.0,-36.47,32.0,X-MC,15.85,55.0,-0.26,1.47,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.0,0.31,1.36,19.15,20.77,45448.0,3182.0,237326.0,-7.46,40.0,X-MC,0.95,62.0,0.07,1.65,8.37,X40N,NTT,BREWERIES
65,SIEMENS,4671.5,0.21,-16.89,51.02,25.51,78908.0,-31435.0,154660.0,-4.98,45.0,H-LC,1.37,49.0,-0.40,1.07,13.97,AR,ATH,ELECTRICAL
20,DABUR,735.0,-0.27,1.90,41.02,43.70,103691.0,4714.0,252782.0,-6.50,67.0,X-MC,1.87,72.0,0.05,1.76,17.24,XY24,BTT,FMCG
12,BERGEPAINT,680.0,-0.01,-5.84,28.65,21.14,61329.0,-13276.0,214063.0,-12.56,41.0,X-MC,2.26,74.0,-0.22,1.49,18.88,XY24,NTT,PAINTS
45,JIOFIN,387.0,-1.56,-4.89,30.88,24.48,78528.0,-13063.0,254302.0,-6.45,46.0,X-LC,2.28,36.0,-0.17,1.77,47.40,XY24,BTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.72,-24.77,108.95,57.19,267298.0,-80787.0,245340.0,-64.40,31.0,X-MC,8.09,58.0,-0.30,1.70,0.00,XY24,NTT,FMCG
43,ITC,452.00,-1.92,-15.93,31.78,10.78,64204.0,-38291.0,202027.0,-49.73,13.0,X-LC,4.25,5.0,-0.60,1.40,0.00,X40,NTT,FMCG
52,PAGEIND,51605.08,-0.68,-10.63,46.15,30.62,97773.0,-25194.0,211860.0,-36.47,32.0,X-MC,15.85,55.0,-0.26,1.47,0.00,X40,ATH,APPARELS
22,DIXON,18931.72,-1.55,-17.74,59.57,31.26,113078.0,-40944.0,189824.0,-61.61,29.0,X-MC,18.00,56.0,-0.36,1.32,0.82,X40N,ATH,IT
11,BATAINDIA,2096.00,-0.90,-38.82,120.98,35.20,95243.0,-49944.0,78726.0,3.25,41.0,X-SC,17.23,92.0,-0.52,0.55,1.11,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,2.31,-26.57,94.53,42.84,45122.0,-17273.0,47733.0,-52.07,56.0,X-SC,29.21,83.0,-0.38,0.33,7.99,XY24,NTT,MISC
58,RELAXO,1176.00,0.05,-47.36,189.33,52.31,144673.0,-68747.0,76413.0,-43.96,49.0,X-SC,7.36,91.0,-0.48,0.53,2.28,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.90,-38.82,120.98,35.20,95243.0,-49944.0,78726.0,3.25,41.0,X-SC,17.23,92.0,-0.52,0.55,1.11,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.09,2.20,21.41,24.08,27778.0,2794.0,129744.0,-6.13,58.0,X-SC,6.97,89.0,0.10,0.90,22.25,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.45,-16.07,70.76,43.32,96729.0,-26172.0,136700.0,-27.94,53.0,X-SC,7.77,90.0,-0.27,0.95,4.96,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.77,-10.89,35.45,20.71,119489.0,-41173.0,337064.0,-23.30,53.0,X-LC,2.65,1.0,-0.34,2.34,12.63,X40,ATH,IT
41,INFY,1972.00,0.16,9.53,22.57,34.24,77346.0,29808.0,342696.0,-14.57,49.0,X-LC,7.45,2.0,0.39,2.38,18.60,X40,NTT,IT
75,TMPV,600.00,-1.18,-24.34,62.54,22.98,106891.0,-54982.0,170916.0,-21.86,58.0,X-LC,4.96,3.0,-0.51,1.19,7.50,XY24,NTT,AUTO
81,VBL,671.64,1.62,0.31,35.25,35.67,111683.0,989.0,316831.0,-11.55,64.0,X-LC,2.37,4.0,0.01,2.20,14.21,X40N,ATH,FMCG
43,ITC,452.00,-1.92,-15.93,31.78,10.78,64204.0,-38291.0,202027.0,-49.73,13.0,X-LC,4.25,5.0,-0.60,1.40,0.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,2.40,-31.96,95.32,32.89,50070.0,-24678.0,52528.0,-748.59,74.0,L-MC,7.24,259.0,-0.49,0.36,44.66,XR,NTT,BANKS
49,MASFIN,398.61,5.43,3.48,17.95,22.05,18199.0,3405.0,101385.0,-10.57,64.0,H-SC,6.61,164.0,0.19,0.70,47.37,XR,ATH,FINANCE
13,BSOFT,831.70,-1.04,-23.36,96.74,50.79,104287.0,-32852.0,107801.0,-1.82,48.0,H-SC,10.70,151.0,-0.32,0.75,25.52,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.36,-16.98,20.48,0.02,29679.0,-29640.0,144919.0,120.98,47.0,M-SC,7.20,234.0,-1.00,1.01,26.05,OX40N,NTT,PAINTS
69,SURYODAY,216.00,0.74,-18.11,50.25,23.03,73684.0,-32436.0,146635.0,60.09,57.0,H-SC,5.35,167.0,-0.44,1.02,45.49,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,2.77,-26.51,53.63,12.90,86117.0,-57920.0,160576.0,136.78,72.0,H-SC,10.98,161.0,-0.67,1.12,33.52,OX40N,NTT,FINANCE
56,RAJOOENG,143.10,9.27,-25.80,88.17,39.61,67053.0,-26450.0,76050.0,-49.88,67.0,H-SC,32.26,135.0,-0.39,0.53,23.10,AR,ATH,MISC
17,COALINDIA,484.83,-0.00,5.11,13.42,19.23,21225.0,7696.0,158156.0,22.81,83.0,L-LC,5.66,179.0,0.36,1.10,27.48,XY25,ATH,MINING
74,TITAGARH,1548.00,-2.56,-18.97,79.85,45.74,156698.0,-45937.0,196240.0,7.85,54.0,H-SC,2.97,174.0,-0.29,1.36,27.63,XY24,NTT,ENGINEERING
82,VOLTAS,1530.00,0.83,16.49,2.75,19.69,6143.0,31623.0,223365.0,5.14,71.0,X-MC,3.61,78.0,5.15,1.55,24.41,XY25,NTT,AC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.10
1,25,44.18
2,50,76.45


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.80
MC    30.97
LC    26.27
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.56
X40      23.86
X40N     12.77
XY25     10.74
XR        8.94
AR        8.67
OX40N     7.55
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.46
H-SC    22.83
X-LC    22.82
M-SC    11.29
X-SC     7.89
H-MC     4.69
M-MC     1.46
M-LC     1.28
L-LC     1.10
H-LC     1.07
L-SC     0.79
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.99,-7.64,42.95
IT,12.65,-17.59,76.90
FINANCE,11.17,-10.95,57.52
MISC,7.06,-27.90,80.21
ELECTRICAL,5.92,-10.12,50.45
PAINTS,5.72,-12.43,29.13
INSURANCE,4.76,0.81,33.13
PHARMA,4.00,-2.23,34.41
AUTO,2.84,-28.48,73.03


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3149825.0,21
AR,1325423.0,10
XR,1205466.0,12
X40,1087608.0,15
X40N,904383.0,9
OX40N,705035.0,10
XY25,348253.0,7
SR,283486.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3510640.0,24
X-MC,1464330.0,16
M-SC,1434169.0,15
X-LC,941524.0,13
X-SC,783686.0,8
H-MC,404378.0,3
L-SC,176983.0,2
M-LC,108246.0,1
H-LC,78908.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1229166.0      6
           AR         910906.0      5
M-SC       XY24       819798.0      6
H-SC       XR         780853.0      7
X-MC       X40        480276.0      7
           XY24       432318.0      3
X-LC       X40        419335.0      6
X-MC       X40N       381789.0      4
M-SC       OX40N      308146.0      5
X-SC       X40N       307585.0      3
H-SC       OX40N      306229.0      4
X-SC       XY24       288104.0      3
H-SC       SR         283486.0      2
X-LC       X40N       215009.0      2
H-MC       AR         209358.0      2
           XY24       195020.0      1
X-SC       X40        187997.0      2
X-LC       XY24       185419.0      2
M-SC       XR         179974.0      2
X-MC       XY25       169947.0      2
M-SC       AR         126251.0      2
X-LC       XY25       121761.0      3
M-LC       XR         108246.0      1
L-SC       OX40N       90660.0      1
           XR          86323.0      1
H-LC       AR          78908.0      1
L-MC       XR          50070.0      1
M-MC       XY25        35320.0      1
L-LC       XY25        21225.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
